In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time 
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

In [3]:
data = pd.read_csv('./data/data.csv')
data.head()

,Unnamed: 0,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,0,What Do We Know about Knowledge Management? Pr...,T. Dingsøyr; F. O. Bjørnson; F. Shull,SINTEF Information and Communication Technolog...,IEEE Software,17 Apr 2009,2009,26.0,3.0,100,...,NaN,28.0,NaN,7.0,NaN,17 Apr 2009,NaN,NaN,IEEE,IEEE Magazines
1,1,Research on National and International Softwar...,G. Li; H. Dong; Q. Zheng; M. Zhou; Y. Guo,"Shandong Comput. Sci. Center, Jinan, China; Sh...",2009 International Conference on Computational...,28 Dec 2009,2009,NaN,NaN,1,...,NaN,NaN,NaN,7.0,NaN,28 Dec 2009,NaN,NaN,IEEE,IEEE Conferences
2,2,Advancing Software Engineering Professional Ed...,M. Ardis; P. Bourque; T. Hilburn; K. Lasfer; S...,Stevens Institute of Technology; École de Tech...,IEEE Software,23 Jun 2011,2011,28.0,4.0,58,...,NaN,11.0,NaN,15.0,NaN,30 Sep 2010,NaN,NaN,IEEE,IEEE Magazines
3,3,Putting Human Aspects of Software Engineering ...,O. Hazzan,Technion–Israel Institute of Technology,IEEE Software,14 Jun 2010,2010,27.0,4.0,90,...,NaN,3.0,NaN,7.0,NaN,14 Jun 2010,NaN,NaN,IEEE,IEEE Magazines
4,4,Where's the Theory for Software Engineering?,P. Johnson; M. Ekstedt; I. Jacobson,KTH Royal Institute of Technology; KTH Royal I...,IEEE Software,21 Aug 2012,2012,29.0,5.0,96,...,NaN,17.0,NaN,NaN,NaN,21 Aug 2012,NaN,NaN,IEEE,IEEE Magazines


In [4]:
data.columns

Index(['Unnamed: 0', 'Document Title', 'Authors', 'Author Affiliations',
       'Publication Title', 'Date Added To Xplore', 'Publication Year',
       'Volume', 'Issue', 'Start Page', 'End Page', 'Abstract', 'ISSN',
       'ISBNs', 'DOI', 'Funding Information', 'PDF Link', 'Author Keywords',
       'IEEE Terms', 'INSPEC Controlled Terms', 'INSPEC Non-Controlled Terms',
       'Mesh_Terms', 'Article Citation Count', 'Patent Citation Count',
       'Reference Count', 'License', 'Online Date', 'Issue Date',
       'Meeting Date', 'Publisher', 'Document Identifier'],
      dtype='object')

In [ ]:
def IntrestInTopic(year):
    databyyear = data[data['Publication Year'] = year]
    

In [5]:
def Count(column,count):
    cnt = Counter()
    for item in data[column]: 
        
        cnt[item] += 1
    return cnt.most_common(count)

#_similarity = cosine_similarity()

In [6]:
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time

#df = data[['Authors','Cites']].groupby('Authors').sum()


#d = df['Cites'].loc[df['Cites'] > 400]
#d.plot.bar()
#print(df['Cites'])



result = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='Article Citation Count',
                        aggfunc=np.sum,
                        fill_value = 0)
#result

In [7]:
#- Publikationstyp: Workshop -> Conference -> Journal 
data['Author Keywords'] = data['Author Keywords'].str.split(';',expand=False)

data['Author Keywords'].head()

data[data['Author Keywords'].str.split(';',expand=False).Contains]
#Mit isin Liste abfragen
#Erst Topic Filtern und dann weiter Arbeiten
    

0    [software engineering, knowledge management, l...
1                                                  NaN
2    [software engineering, computer and informatio...
3    [human aspects of software engineering, softwa...
4    [software engineering theory, theory, explanat...
Name: Author Keywords, dtype: object

In [8]:
result

Publication Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author Keywords,,,,,,,,,,,,,
- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management;Requirements/specifications;initiation and scope definition;management,0,0,0,0,0,0,0,12,0,0,0,0,0
11th International Symposium on Empirical Software Engineering and Measurement;13th International Conference on Predictive Models and Data Analytics in Software Engineering;21st International Systems and Software Product Line Conference;empirical software engineering;predictive models;data analytics;software product lines;product line engineering;developer productivity;issue resolution;agile development;microservices;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
33rd International Conference on Software Maintenance and Evolution;ICSME 17;32nd International Conference on Automated Software Engineering;ASE 17;software maintenance;software evolution;automated software engineering;flaky tests;technical debt;self-admitted technical debt;SATD;QA bots;regular expressions;regexes;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
40th International Conference on Software Engineering;17th International Conference on Software Reuse;gender;gender stereotypes;GenderMag;sentiment;sentiment analysis;Stack Overflow;Senti4SD;software product lines;SPL;software product line engineering;SPLE;variability management;safety-critical systems;software engineering education;software reuse;software development;software engineering;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,1,0,0,0
API Documentation;Code Sample;Usage Scenario;Stack Overflow;Crowd knowledge,0,0,0,0,0,0,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
vulnerability detection;smart fuzzing;automated testing;file format;grammar;input structure,0,0,0,0,0,0,0,0,0,0,9,0,0
waterfall model;software development;software engineering;convenience sampling,0,0,0,0,0,0,0,0,0,0,0,0,0
wireless technology;machine-to-machine;smart factory;software engineering;embedded systems;green computing,0,0,0,0,0,16,0,0,0,0,0,0,0
